In [6]:
import os
import openpyxl
import re
import pandas as pd

# Define a function to remove all leading and trailing whitespace
def remove_whitespace(text):
    return re.sub(r'^\s+|\s+$', '', text)

# Function to extract information from an Excel file
def extract_info_from_excel(file_path):
    data = []
    max_data = []  # To store data from the sheet with the most information
    max_data_length = 0  # To keep track of the length of max_data
    try:
        # Use pandas to read XLS file with multiple sheets
        xls = pd.ExcelFile(file_path)

        # Iterate over all sheet names
        for sheet_name in xls.sheet_names:
            print(f"Processing sheet: {sheet_name}")
            df = pd.read_excel(xls, sheet_name, header=None)
            #print(df)

            # Flag to track whether data was found in the sheet
            data_found_in_sheet = False

            # Find the header row using the keywords
            header_row_index = None
            for i, row in enumerate(df.values):
                for j, cell_value in enumerate(row):
                    if cell_value is not None and any(keyword in str(cell_value).lower() for keyword in
                                                      ["artigo", "cap", "capitulo", "codigo", "descrição",
                                                       "designação"]):
                        header_row_index = i
                        break
                if header_row_index is not None:
                    break

            if header_row_index is None:
                print(f"No header row found in {sheet_name}")
                continue  # Skip this sheet and move to the next one

            header_row = df.iloc[header_row_index]
            print("-------")
            print(header_row)

            code_col_index = None
            description_col_index = None
            unit_col_index = None
            pattern_a = r'\b(?!\s)(?:art|artº.|artº|cap|cod|item|codigo|artigo|código)\b(?!\w)'
            pattern_b= r'\b(?:un|unidade|un.)\b'

            for i, header in enumerate(header_row):
                if isinstance(header, str):  # Check if header is a string
                    header = remove_whitespace(header)
                    header = re.sub(r'[\t\x0B\x0C\x00-\x08\x0E-\x1F\x7F-\x9F]+', ' ', header)
                    if code_col_index is None:  # Check if code_col_index is not assigned yet
                        match = re.search(pattern_a, header.lower(), flags=re.IGNORECASE)
                        if match:
                            code_col_index = i
                    
                    if header and any(keyword in str(header).lower() for keyword in ["descrição", "designação"]):
                        description_col_index = i
                    
                    if re.search(pattern_b, header.lower(), flags=re.IGNORECASE):
                        unit_col_index = i

            
            # Check the values of code_col_index, description_col_index, and unit_col_index here
            print("Code Column Index:", code_col_index)
            print("Description Column Index:", description_col_index)
            print("Unit Column Index:", unit_col_index)
            code_list = []
            description_list = []
            unit_list = []
            extracted_data = []

            if unit_col_index is not None:
                for index, row in df.iloc[header_row_index+1:].iterrows():
                    code = row[code_col_index] if code_col_index is not None else ""
                    description = row[description_col_index] if description_col_index is not None else ""
                    unit = row[unit_col_index]
                    if sum(1 for item in [code, description, unit] if pd.notna(item) and item != "") >= 2:
                            extracted_data.append((code, description, unit))
            else:
                for index, row in df.iloc[header_row_index+1:].iterrows():
                    code = row[code_col_index] if code_col_index is not None else ""
                    code_list.append(code)
                    description = row[description_col_index] if description_col_index is not None else ""
                    description_list.append(description)
                
                pattern_unit = r"\(\w{1,6}\)"
                unit_list = []
                for text in description_list: 
                    text = str(text)
                    found = re.findall(pattern_unit, text)
                    if not found:
                        unit_list.append("")
                    else:
                        unit_list.extend(found)

                data_tuple = [(a, b, c) if i < len(unit_list) - 1 else (a, b, '') for i, (a, b, c) in enumerate(zip(code_list, description_list, unit_list[1:]))]
                filtered_data_tuple = [tup for tup in data_tuple if sum(1 for item in tup if item != '') > 2]
                extracted_data.extend(filtered_data_tuple)
            
            # Append the extracted data to the result list
            if extracted_data:
                data.extend(extracted_data)
                data_found_in_sheet = True  # Set the flag to True if data was found

            # Compare the length of data with max_data
            if len(data) > max_data_length:
                max_data = data
                max_data_length = len(data)
            
            # Print the message based on the flag value
            if data_found_in_sheet:
                print(f"Data found in sheet: {sheet_name}")
            else:
                print(f"No data found in sheet: {sheet_name}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    return data


# Folder containing Excel files
folder_path = r"C:\Users\Luís\OneDrive - Universidade do Porto\Scrape de MQT data treatment experiments\All MQT Data"

#file_path = r"C:\Users\Luís\OneDrive - Universidade do Porto\Scrape de MQT data treatment experiments\All MQT Data\00.Med_Orç_Requalif_Ruas_Escolas_F.xls"
#extract_info_from_excel(file_path)

# Create an empty DataFrame to store the scraped data
result_df = pd.DataFrame(columns=["Art.", "Description", "Unit"])

# Loop through all files in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith((".xlsx", ".xls")):
        file_path = os.path.join(folder_path, filename)
        extracted_data = extract_info_from_excel(file_path)
        
        # Append the extracted data to the result DataFrame
        if extracted_data:
            temp_df = pd.DataFrame(extracted_data, columns=["Art.", "Description", "Unit"])
            result_df = result_df.append(temp_df, ignore_index=True)

# Save the result DataFrame to a CSV file
result_df.to_excel("extracted_data.xlsx", index=False)

print("Extraction complete. Data saved to extracted_data.csv")







Processing sheet: MQT
-------
0             Tipo (1)
1             Art. (2)
2        Descrição (3)
3               Un (4)
4    Total (11)       
5       Preço Unitário
6        Preço Parcial
7                Total
Name: 8, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 3
Data found in sheet: MQT
Processing sheet: MQT
-------
0             Tipo (1)
1             Art. (2)
2        Descrição (3)
3               Un (4)
4    Total (11)       
5       Preço Unitário
6        Preço Parcial
7                Total
Name: 9, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 3
Data found in sheet: MQT
Processing sheet: ESTIMATIVA ORÇAMENTAL
-------
0             Tipo (1)
1             Art. (2)
2        Descrição (3)
3               Un (4)
4    Total (11)       
5       Preço Unitário
6        Preço Parcial
7                Total
Name: 8, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 3
Data found 

C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


Processing sheet: Resumo
No header row found in Resumo
Processing sheet: MQT
-------
0           Artº Nº
1        Designação
2                Un
3            Quant.
4    Preço unitário
5       Importância
6    Total Capítulo
Name: 13, dtype: object
Code Column Index: 0
Description Column Index: 1
Unit Column Index: 2
Data found in sheet: MQT
Processing sheet: MEDICOES
-------
0           NaN
1    DESIGNAÇÃO
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
Name: 6, dtype: object
Code Column Index: None
Description Column Index: 1
Unit Column Index: None
No data found in sheet: MEDICOES


C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


Processing sheet: MQUANT
-------
0                        NaN
1    DESCRIÇÃO DOS TRABALHOS
2                        Un.
3                        NaN
4                  ORÇAMENTO
5                        NaN
Name: 5, dtype: object
Code Column Index: None
Description Column Index: 1
Unit Column Index: 2
Data found in sheet: MQUANT
Processing sheet: RESUMO
-------
0                        Cap.
1    Designação dos Capítulos
2                  Sub-Totais
3                      Totais
Name: 10, dtype: object
Code Column Index: 0
Description Column Index: 1
Unit Column Index: None
No data found in sheet: RESUMO
Processing sheet: MAPA QUANTIDADES
-------
0        Artigo
1    DESIGNAÇÃO
2           NaN
3           NaN
4     ORÇAMENTO
5           NaN
Name: 9, dtype: object
Code Column Index: 0
Description Column Index: 1
Unit Column Index: None
Data found in sheet: MAPA QUANTIDADES
Processing sheet: templet
-------
0          Nível
1    Cód. Artigo
2      Descrição
3             Qt
4        Unid

...

Data found in sheet: Mapa Quantidades
Processing sheet: Mapa de Medições
-------
0          Nível
1    Cód. Artigo
2      Descrição
3             Qt
4        Unidade
Name: 0, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 4
Data found in sheet: Mapa de Medições
Processing sheet: Folha1
-------
0                        Nível
1                  Cód. Artigo
2                    Descrição
3                           Qt
4                      Unidade
5          Preço de Referência
6    Total Preço de Referência
7     Comentários do Comprador
Name: 0, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 4
Data found in sheet: Folha1
Processing sheet: MTQ
-------
0                        Nível
1                  Cód. Artigo
2                    Descrição
3                           Qt
4                      Unidade
5          Preço de Referência
6    Total Preço de Referência
7     Comentários do Comprador
Name: 0, dtype: object


C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


Processing sheet: Folha Unica
-------
0                     Artigo
1    DESCRIÇÃO DOS TRABALHOS
2                     Totais
3                         Un
4                        NaN
5                        NaN
6             Preço Unitário
7    Preço    do      Artigo
8          Total do Capítulo
Name: 3, dtype: object
Code Column Index: 0
Description Column Index: 1
Unit Column Index: 3
Data found in sheet: Folha Unica
Processing sheet: ESTIMATIVA ORÇAMENTAL
-------
0             Tipo (1)
1             Art. (2)
2        Descrição (3)
3               Un (4)
4    Total (11)       
5       Preço Unitário
6        Preço Parcial
7                Total
Name: 8, dtype: object
Code Column Index: 1
Description Column Index: 2
Unit Column Index: 3
Data found in sheet: ESTIMATIVA ORÇAMENTAL


C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)
C:\Users\Luís\AppData\Local\Temp\ipykernel_23536\404097425.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(temp_df, ignore_index=True)


Extraction complete. Data saved to extracted_data.csv
